In [67]:
import os 
import pandas as pd 
import psycopg2
import requests
from bs4 import BeautifulSoup
import time

url_base = 'https://www.index.hr/oglasi/brodski-motori/gid/3569?pojamZup=-2&tipoglasa=1&sortby=1&elementsNum=100&grad=0&naselje=0&cijenaod=0&cijenado=115000&num='
data = []

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="sibin",
    user="sibinuser",
    password="sib123"
)
# Create a cursor object to execute SQL queries
cur = conn.cursor()

for page_num in range(1, 8):
    url = url_base + str(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    ads = soup.find_all('div', class_='OglasiRezHolder')
    if len(ads) == 0:
        break
    for i, ad in enumerate(ads, 1):
        print("Starting to process ads\n")
        print(f"Procesing {i}, ad HTML content looks like this, lets get data after this:\n", ad)
        classes = ad.get('class', [])
        if len(classes) == 1 and 'OglasiRezHolder' in classes:
            print(f'\nProcessing {i}/{len(ads)}...')
            title = ad.find('span', class_=lambda c: c.startswith('title')).text.strip()
            print(f'Listing title: {title}')
            img_tag = ad.find('img', class_='adexElement adexFoundElement-0')
            img_url = img_tag['src'] if img_tag else ''
            print(f'Image: {img_url}')
            description = ad.find('span', class_='lead').text.strip()
            print(f'Info: {description}')
            county_element = ad.find('li', class_='icon-marker')
            county = county_element.text.strip() if county_element else ''
            print(f'County: {county}')
            published_ago = ad.find('li', class_='icon-time').text.strip()
            print(f'Published ago: {published_ago}')
            price_EUR = ad.find('span', class_='price').find('span').text.strip()
            print(f'Price EUR: {price_EUR}')
            price_kn = ad.find('span', class_='price').text.strip().split('~')[1].strip()
            print(f'Price kn: {price_kn}')
            ad_url = ad.find('a', class_='result')['href']
            print(f'Link: {ad_url}\n')
            
               # Insert the data into the PostgreSQL database
            cur.execute("INSERT INTO ads (title, image, info, county, published_ago, price_eur, price_kn, ad_url) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                        (title, img_url, description, county, published_ago, price_EUR, price_kn, ad_url))
            conn.commit()

            data.append({
                'Title': title,
                'Image': img_url,
                'Info': description,
                'County': county,
                'Published_ago': published_ago,
                'Price_EUR': price_EUR,
                'Price_kn': price_kn,
                'Link': ad_url
            })
        else:
            print(f'\nSkipping ad {i}/{len(ads)}...')
            # If word "Banner" is in name of HTML class 
            if any('Banner' in c for c in classes):
                print('This is an advertisement and not an ad.')
            else:
                other_classes = [c for c in classes if c != 'OglasiRezHolder']
                print(f'Classes: {other_classes}')
    time.sleep(2)  # delay for 2 seconds before making next request
    
    # Close the cursor and connection objects
cur.close()
conn.close()

df = pd.DataFrame(data)
display(df)

Starting to process ads

Procesing 1, ad HTML content looks like this, lets get data after this:
 <div class="OglasiRezHolder">
<a class="result" href="https://www.index.hr/oglasi/mercruiser-5-7-hitno/oid/3998807">
<figure class="result_photo" data-x="2">
<img alt="Mercruiser 5.7 *HITNO*" src="https://www.index.hr/oglasi/userdocsimages/oglas/_2023//4//28/3998807/3a92ff67-6a69-4609-a189-1e17bf305f9a-280420230928549498.jpeg?preset=oglas-slika-view"/>
</figure>
<span class="result_body">
<span class="head">
<span class="title px18">
Mercruiser 5.7 *HITNO*
<span class="addFavorit icon-heart tooltip_bottom" title="Spremi oglas">
</span>
</span>
<span class="lead" data-fff="bb"></span>
</span>
<span class="foot">
<ul class="info">
<li class="icon-marker">Šibensko-kninska</li>
<li class="icon-time">Objavljeno prije 6 h</li>
</ul>
<span class="price"><span>2.600 €</span> ~ 19.590 kn</span></span>
</span>
</a>
</div>

Processing 1/111...
Listing title: Mercruiser 5.7 *HITNO*
Image: 
Info: 
Coun

,Title,Image,Info,County,Published_ago,Price_EUR,Price_kn,Link
0,Mercruiser 5.7 *HITNO*,,,Šibensko-kninska,Objavljeno prije 6 h,2.600 €,19.590 kn,https://www.index.hr/oglasi/mercruiser-5-7-hit...
1,Yamaha 15KS,,,Primorsko-goranska,Objavljeno prije 6 h,1.300 €,9.795 kn,https://www.index.hr/oglasi/yamaha-15ks/oid/39...
2,YAMAHA 60 KS NOVO - NA LAGERU!,,,Primorsko-goranska,Objavljeno prije 6 h,7.429 €,55.974 kn,https://www.index.hr/oglasi/yamaha-60-ks-novo-...
3,YAMAHA 115 KS AETX U PARU SA KOMPLET OPREMOM,,,Primorsko-goranska,Objavljeno prije 6 h,15.000 €,113.018 kn,https://www.index.hr/oglasi/yamaha-115-ks-aetx...
4,Yamaha 20 KS NOVO - NA LAGERU!,,,Primorsko-goranska,Objavljeno prije 6 h,5.119 €,38.569 kn,https://www.index.hr/oglasi/yamaha-20-ks-novo-...
...,...,...,...,...,...,...,...,...
695,"Prodajem motor za barku Lombardini dizel, 1 ci...",,"Prodajem motor za barku Lombardini dizel, 1 ci...",Primorsko-goranska,Objava 09.03.2019.,"1,36 €",10 kn,https://www.index.hr/oglasi/prodajem-motor-za-...
696,Tomos,,Tomos 4 u odlicnom stanju,Šibensko-kninska,Objava 01.03.2019.,136 €,1.000 kn,https://www.index.hr/oglasi/tomos/oid/1624519
697,YANMAR 8,,,Zadarska,Objava 15.02.2019.,2.000 €,14.752 kn,https://www.index.hr/oglasi/yanmar-8/oid/1603752
698,Brodski motor HAC 7 PS,,Prodajem uređeni i potpuno ispravan HAC 7 konj...,Primorsko-goranska,Objava 03.12.2018.,1.000 €,7.532 kn,https://www.index.hr/oglasi/brodski-motor-hac-...


In [73]:
df.to_csv('raw_data.csv', index=False)
df.to_excel('raw_data.xlsx', index=False)